In [ ]:
import pandas as pd
import os
import sys
import numpy as np
import csv
import pickle
import string
import mtscluster
from tslearn.metrics import dtw, dtw_path
from sklearn.preprocessing import MinMaxScaler
import kmedoids
import matplotlib.pyplot as plt 
import ruptures as rpt  
from ruptures.metrics import hausdorff
import copy

In [ ]:
Folder_Path = r'../datafolder'     
SaveFile_Path =  r'melped'       
SaveFile_Name = r'meldata19.csv'              

In [ ]:
os.chdir(Folder_Path)
file_list = os.listdir()

df4 = pd.read_csv(Folder_Path +'/'+'April_2019.csv')   
df5 = pd.read_csv(Folder_Path +'/'+'May_2019.csv')
df6 = pd.read_csv(Folder_Path +'/'+'June_2019.csv')
df7 = pd.read_csv(Folder_Path +'/'+'July_2019.csv')
df8 = pd.read_csv(Folder_Path +'/'+'August_2019.csv')
df9 = pd.read_csv(Folder_Path +'/'+'September_2019.csv')
df10 = pd.read_csv(Folder_Path +'/'+'October_2019.csv')
df11 = pd.read_csv(Folder_Path +'/'+'November_2019.csv')
df12 = pd.read_csv(Folder_Path +'/'+'December_2019.csv')

In [ ]:
temp = pd.concat([df4, df5,df6,df7,df8,df9,df10,df11,df12], join="inner")

In [ ]:
coldel = []
for i,x in enumerate(temp.columns):
    if len(temp[temp[x]==-1])>66:
        coldel.append(x)
print(len(coldel))

In [ ]:
dfnow = temp.drop(coldel,axis=1)

In [ ]:
dfnan = dfnow.replace(-1, np.nan)

In [ ]:
coldel2 = []
for i,x in enumerate(dfnan.columns):
    for j in range(dfnan.shape[0]):
        if dfnan.iat[j,i] == "undefined":
            coldel2.append(x)
            break
coldel2.append('Collins St (North)')
print(coldel2)

In [ ]:
dfnan = dfnan.drop(coldel2,axis=1)

In [ ]:
dfnan = dfnan.reset_index(drop=True)

In [ ]:
colname = list(dfnan.columns)
streetname = colname[2:]
mtd = ['mean']*(len(streetname))
dict1 = dict(zip(streetname,mtd))
print('len(streetname): ',len(streetname))

In [ ]:
meanbyhour = dfnan.groupby('Hour').agg(dict1)

In [ ]:
for column in streetname:
    for i in range(dfnan.shape[0]):
        if np.isnan(dfnan[column][i]):
            dfnan.loc[i,column] = meanbyhour.loc[(i%24),column]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dfarray = dfnan.iloc[:,2:].to_numpy()
scaler = MinMaxScaler()
dfarray = scaler.fit_transform(dfarray)
weeks = []
for i in range(len(dfarray)//(24*7)):
    weeks.append(dfarray[i*(24*7):(i+1)*(24*7),:])
Train_data = np.array(weeks)

tsdistances = np.zeros((np.shape(Train_data)[0],np.shape(Train_data)[0]))
for ind,i in enumerate(Train_data):
    for c_ind,j in enumerate(Train_data):
        cur_dist = 0.0
        #Find sum of distances along each dimension
        for z in range(np.shape(Train_data)[2]):
            cur_dist += dtw(i[:,z],j[:,z])
        tsdistances[ind,c_ind] = cur_dist
#print('Distances calculated: ', tsdistances)
#pd.DataFrame(tsdistances)

In [ ]:
cluster_result =  kmedoids.pam(tsdistances, 2, max_iter=1000, init='random', random_state=None)
cluster_result.labels

In [ ]:
clsdisframe = pd.DataFrame(tsdistances)

In [ ]:
dis_first = []
dis_second = []

for i in range(len(tsdistances)):
    if cluster_result.labels[i] == 1:
        dis_second.append(clsdisframe.iloc[i,cluster_result.medoids[1]])
    else:
        dis_first.append(clsdisframe.iloc[i,cluster_result.medoids[0]])

In [ ]:
dropedge1 = np.percentile(np.array(dis_first), 75)+1.5*(np.percentile(np.array(dis_first), 75)-np.percentile(np.array(dis_first), 25))

In [ ]:
dropedge2 = np.percentile(np.array(dis_second), 75)+1.5*(np.percentile(np.array(dis_second), 75)-np.percentile(np.array(dis_second), 25))

In [ ]:
medo_iter = []
for i in range(1000):
    clusters, curr_medoids = mtscluster.cluster(tsdistances, 2)
    # print('Mediods are :')
    # print(curr_medoids)
    # print('Cluster assigments : ')
    # print(clusters)
    medo_iter.append(curr_medoids)
medo_iter = [l.tolist() for l in medo_iter]
for l in medo_iter:
    l.sort()

In [ ]:
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num
 
List = medo_iter
clsdistance = pd.DataFrame(tsdistances).iloc[:,most_frequent(List)]
clsdistance['sum'] = clsdistance.iloc[:,0]+clsdistance.iloc[:,1]
clsdistance = clsdistance.sort_values(by=['sum'])

In [ ]:
dissum = clsdistance['sum'].to_numpy() 
dropedge = np.percentile(dissum, 75)+1.5*(np.percentile(dissum, 75)-np.percentile(dissum, 25))
weekdrop = clsdistance[clsdistance['sum']>dropedge].index.to_list()
dfgood = dfnan.iloc[4*24*7:38*24*7,:].reset_index(drop=True)

In [ ]:
df_initial = copy.deepcopy(dfgood.iloc[:168,2:])
df_initial

In [ ]:
for i in range(1,27):   
    df_initial+=dfgood.iloc[i*168:(i+1)*168,2:].reset_index(drop=True)

In [ ]:
df_initial = df_initial/27

In [ ]:
dfcomplete = copy.deepcopy(dfgood)

In [ ]:
for i in range(int(len(dfcomplete)/168)):
    dfcomplete.iloc[i*168:(i+1)*168,2:] = dfcomplete.iloc[i*168:(i+1)*168,2:].reset_index(drop=True)-df_initial

In [ ]:
dfcenter = dfnan.iloc[7*24*7:8*24*7,:].reset_index(drop=True)

In [ ]:
dfcenterall = dfcenter.iloc[:,2:].to_numpy()
scaler = MinMaxScaler()
dfcentertrain = scaler.fit_transform(dfcenterall)

tsdistancesall = np.zeros((np.shape(dfcentertrain)[1],np.shape(dfcentertrain)[1]))
for i in range(dfcentertrain.shape[1]):
    for j in range(dfcentertrain.shape[1]):
        tsdistancesall[i,j] = dtw(dfcentertrain[:,i],dfcentertrain[:,j])  
#print('Distances calculated: ', tsdistances)
#pd.DataFrame(tsdistances)

In [ ]:
dfday = np.zeros([24,30])
for i in range(7):
    dfday = dfday+dfcenter.iloc[:,2:].iloc[i*24:(i+1)*24,:].to_numpy()
dfday = dfday/7

In [ ]:
locationdata = pd.read_csv('../datafolder/Pedestrian_Counting_System_-_Sensor_Locations.csv')
locationdata = locationdata[['sensor_description','latitude','longitude']]
locationdata.set_index('sensor_description', inplace=True)
locationdata = locationdata.T

In [ ]:
streetdata = locationdata[streetname] 

In [ ]:
stationdata = pd.read_csv('/data/dyw/melped/Pedestrian_Counting_System_-_Sensor_Locations.csv')
stationdata = stationdata[['sensor_description','latitude','longitude']]
stationdata.set_index('sensor_description', inplace=True)
stationdata = stationdata.T

stationdf = pd.concat([stationdata['Southern Cross Station'],stationdata['Flinders Street Station Underpass'],stationdata['Flagstaff Station'],stationdata['Melbourne Central'],stationdata['Collins Place (South)']],axis=1)
staname = list(stationdf.columns)

In [ ]:
stationdistmat = pd.DataFrame(np.zeros((len(staname),len(streetname))))

In [ ]:
for i,sta in enumerate(staname):
    for j,stre in enumerate(streetname):
        stationdistmat.iloc[i,j]=(np.sqrt((streetdata[stre][0]-stationdf[sta][0])**2+(streetdata[stre][1]-stationdf[sta][1])**2))

In [ ]:
stationminmat =  stationdistmat.min()

In [ ]:
stadistlist = []
stasensorfrom = []
stasensorto = []
for ifrom,istree in enumerate(streetname):
    for jto,jstree in enumerate(streetname):
        stasensorfrom.append(istree)
        stasensorto.append(jstree)
        stadistlist.append(np.abs((stationminmat[ifrom]-stationminmat[jto])))
# print(sensorfrom)
# print(sensorto)
# print(distlist[31])
stadistancedict = {'from':stasensorfrom,'to':stasensorto, 'dist': stadistlist}
stadistance_df0 = pd.DataFrame(stadistancedict)

In [ ]:
tsdistlist = []
tssensorfrom = []
tssensorto = []
for ifrom,istree in enumerate(streetname):
    for jto,jstree in enumerate(streetname):
        tssensorfrom.append(istree)
        tssensorto.append(jstree)
        tsdistlist.append(tsdistancesall[ifrom,jto])
# print(sensorfrom)
# print(sensorto)
# print(distlist[31])
tsdistancedict = {'from':tssensorfrom,'to':tssensorto, 'dist': tsdistlist}
tsdistance_df0 = pd.DataFrame(tsdistancedict)

In [ ]:
distlist = []
sensorfrom = []
sensorto = []
for ifrom in streetname:
    for jto in streetname:
        sensorfrom.append(ifrom)
        sensorto.append(jto)
        distlist.append(np.sqrt((streetdata[ifrom][0]-streetdata[jto][0])**2+(streetdata[ifrom][1]-streetdata[jto][1])**2))
# print(sensorfrom)
# print(sensorto)
# print(distlist[31])
distancedict = {'from':sensorfrom,'to':sensorto, 'dist': distlist}
distance_df0 = pd.DataFrame(distancedict)

In [ ]:
def get_adjacency_matrix(distance_df, sensor_ids, normalized_k=0.1):
    """
    :param distance_df: data frame with three columns: [from, to, distance].
    :param sensor_ids: list of sensor ids.
    :param normalized_k: entries that become lower than normalized_k after normalization are set to zero for sparsity.
    :return:
    """
    num_sensors = len(sensor_ids)
    dist_mx = np.zeros((num_sensors, num_sensors), dtype=np.float32)
    dist_mx[:] = np.inf
    # Builds sensor id to index map.
    sensor_id_to_ind = {}
    for i, sensor_id in enumerate(sensor_ids):
        sensor_id_to_ind[sensor_id] = i

    # Fills cells in the matrix with distances.
    for row in distance_df.values:
        if row[0] not in sensor_id_to_ind or row[1] not in sensor_id_to_ind:
            continue
        dist_mx[sensor_id_to_ind[row[0]], sensor_id_to_ind[row[1]]] = row[2]

    # Calculates the standard deviation as theta.
    distances = dist_mx[~np.isinf(dist_mx)].flatten()
    std = distances.std()
    adj_mx = np.exp(-np.square(dist_mx / std))
    # Make the adjacent matrix symmetric by taking the max.
    # adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])

    # Sets entries that lower than a threshold, i.e., k, to zero for sparsity.
    adj_mx[adj_mx < normalized_k] = 0
    return sensor_ids, sensor_id_to_ind, adj_mx


In [ ]:
# distance matrix based on latitude and longtitude
sensor_ids0, sensor_id_to_ind0, adj_mx0 = get_adjacency_matrix(distance_df0,streetname)

In [ ]:
# distance matrix based on time series similarity
tmsensor_ids0, tmsensor_id_to_ind0, tm_adj_mx0 = get_adjacency_matrix(tsdistance_df0,streetname)

In [ ]:
# # distance matrix based on minimum distance to train station
# stasensor_ids0, stasensor_id_to_ind0, sta_adj_mx0 =  get_adjacency_matrix(stadistance_df0,streetname)

In [ ]:
adjmat0001 = adj_mx0 + 0.001*tm_adj_mx0
adjmat001 = adj_mx0 + 0.01*tm_adj_mx0
adjmat002 = adj_mx0 + 0.02*tm_adj_mx0
adjmat005 = adj_mx0 + 0.05*tm_adj_mx0
adjmat01 = adj_mx0 + 0.1*tm_adj_mx0
adjmat02 = adj_mx0 + 0.2*tm_adj_mx0
adjmat05 = adj_mx0 + 0.5*tm_adj_mx0
adjmat1 = adj_mx0 + tm_adj_mx0
adjmat2 = adj_mx0 + 2*tm_adj_mx0
adjmat5 = adj_mx0 + 5*tm_adj_mx0

In [ ]:
with open('adjmat.pkl', 'wb') as f:
    pickle.dump([sensor_ids0, sensor_id_to_ind0, adj_mx0], f, protocol=2)

In [ ]:
with open('adjmat0001.pkl', 'wb') as f:
    pickle.dump([sensor_ids0, sensor_id_to_ind0, adjmat0001], f, protocol=2)

In [ ]:
with open('adjmat001.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat001], f, protocol=2)

In [ ]:
with open('adjmat002.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat002], f, protocol=2)

In [ ]:
with open('adjmat005.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat005], f, protocol=2)

In [ ]:
with open('adjmat01.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat01], f, protocol=2)

In [ ]:
with open('adjmat02.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat02], f, protocol=2)

In [ ]:
with open('adjmat05.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat05], f, protocol=2)

In [ ]:
with open('adjmat1.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat1], f, protocol=2)

In [ ]:
with open('adjmat2.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat2], f, protocol=2)

In [ ]:
with open('adjmat5.pkl', 'wb') as f:
    pickle.dump([tmsensor_ids0, tmsensor_id_to_ind0, adjmat5], f, protocol=2)

In [ ]:
dfgood['Hour'] = dfgood['Hour'].apply(str)
dfgood['Time'] = dfgood['Date']+' '+dfgood['Hour']
dfgood2=dfgood[['Time']+streetname]
dfgood2['Time'] = pd.to_datetime(dfgood2['Time'], format="%d/%m/%Y %H")
dfgoodclean = dfgood2[streetname]
dfgoodclean.index = list(dfgood2['Time'])

In [ ]:
dfgoodclean.to_hdf(f'meldatagood.h5', key='dfgoodclean')

In [ ]:
# For the DCRNN part, see https://github.com/xlwang233/pytorch-DCRNN 